<a href="https://colab.research.google.com/github/F1ameX/Modern-Methods-of-Deep-Machine-Learning/blob/main/1_polynominal_regression/1_polynominal_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
import math
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt

In [83]:
N = 250 # size of sample
sample = np.empty((N, 2), dtype = float)
epsilons = [0.001, 0.1, 1, 1.5] # epsilon variability
rng = np.random.default_rng() # random generator
a, b, c, d = rng.uniform(low = -3.0, high = 3.0, size = 4) # random coefficient

In [84]:
def f(x : float, option : int = 1):
  global a, b, c, d

  if option == 0:
    return a * np.pow(x, 3) + b * np.pow(x, 2) + c * x + d

  elif option == 1:
    return x * np.sin(2 * np.pi * x)

In [104]:
def draw_graph(title : str, X : np.array, y_original : np.array, y_noised : np.array) -> None:
    plt.figure(figsize = (9, 4.5))
    plt.title(title)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.xlim(-1, 1)
    plt.ylim(-3, 3)
    plt.scatter(X, y_noised, c = 'red', linewidths = 0.25, alpha = 0.75)
    plt.scatter(X, y_original, c = 'blue', linewidths = 0.25)
    plt.grid(True)
    plt.show()

In [ ]:
for i in range(N):
    sample[i][0] = rng.uniform(low = -1.0, high = 1.0)

for func in range(2):
    clear = f(sample[:, 0], option = func)
    for distribution in range(2):
        for eps in epsilons:
            for i in range(N):

                if distribution == 0:
                    sample[i][1] = f(sample[i][0], option = func) + rng.uniform(low = -eps, high = eps)
                else:
                    sample[i][1] = f(sample[i][0], option = func) + np.clip(rng.normal(loc = 0.0, scale = eps / 3), -eps, eps)

            if func == 0:
                title = 'polynom'
            if func == 1:
                title = 'sin'

            if distribution == 0:
                title += '_uniform'
            if distribution == 1:
                title += '_normal'

            title += f"_epsilon={eps}"
            draw_graph(title, sample[:, 0], clear, sample[:, 1])

In [124]:
class PolynomicalRegression():
    def __init__(self, degree : int) -> None:
        self.degree = degree
        self.weights = None

    def _design_matrix(self, X : np.array) -> None:
        wandermond_matrix = np.ones(X.shape[0], dtype=float).reshape(-1, 1)

        for i in range(1, self.degree + 1):
            wandermond_matrix = np.c_[wandermond_matrix, X ** i]

        return wandermond_matrix

    def _loss(self, y_true : np.array, y_score : np.array) -> float:
        return np.mean((y_true - y_score) ** 2)

    def fit(self, X: np.array,
            y : np.array,
            n_iterations : int = 1000,
            learning_rate : float = 0.01,
            verbose : int = 100) -> np.array:

        self.weights = np.random.randn(self.degree + 1, 1)
        X_poly = self._design_matrix(X)

        for iter in range(n_iterations):
            y_pred = X_poly @ self.weights
            loss = self._loss(y, y_pred)
            if iter % verbose == 0:
                print(loss)
            grad = 2 / X.shape[0] * X_poly.T @ (y_pred - y)

            self.weights = self.weights - learning_rate * grad

        return self.weights

    def predict(self, X: np.array) -> np.array:
        if self.weights is not None:
            X_poly = self._design_matrix(X)
            return X_poly @ self.weights

In [125]:
def split_data(X: np.array, y: np.array, ratio : float = 0.33, random_state : int = 42):
    rng = np.random.default_rng(seed = random_state)
    idx = np.arange(X.shape[0])
    rng.shuffle(idx)
    left_share = int(ratio * X.shape[0])

    X_left = X[idx[: left_share]]
    y_left = y[idx[: left_share]]
    X_right = X[idx[left_share: ]]
    y_right = y[idx[left_share: ]]

    return X_left, y_left, X_right, y_right

In [126]:
X_train, y_train, X_test, y_test = split_data(sample[:, 0], sample[:, 1])

In [ ]:
for m in range(21):
    model = PolynomicalRegression(degree = m)
    model.fit(X_train, y_train, learning_rate = 0.1, n_iterations = 300, verbose = 150)
    y_pred = model.predict(X_test)